In [1]:
import tensorflow as tf

import numpy as np
import os
import tensorflow as tf
import shutil
import random
import cv2
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.applications.vgg16 import preprocess_input, VGG16
from keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras_applications.imagenet_utils import _obtain_input_shape
from keras import backend as K
from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.python.keras.utils.data_utils import get_file

from tensorflow.keras.utils import get_source_inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input

In [2]:
tf.config.list_physical_devices('GPU')

[]

## Preprocessing

In [3]:
class Preprocessing:

    def load_data(input_directory,
                  train_directory,
                  input_img_size: int = 64,
                  mode_kelas="sparse"):

        CATEGORY = ['training', 'validation']
        data_generator = ImageDataGenerator(rescale=1/255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')
        train_dataset = data_generator.flow_from_directory(train_directory, (input_img_size, input_img_size), class_mode=mode_kelas)

        output = []

        for category in CATEGORY:
            path = os.path.join(input_directory, category)
            print(path)
            images = []
            labels = []

            for folder in os.listdir(path):
                label = folder

                for file in os.listdir(os.path.join(path, folder)):
                    img_path = os.path.join(os.path.join(path, folder), file)
                    image = cv2.imread(img_path)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    image = cv2.resize(image, (input_img_size, input_img_size))
                    images.append(image)
                    labels.append(label)

            images = np.array(images)
            labels = np.array(labels)
            output.append((images, labels))
            return output

    def datagen(input_directory, input_img_size: int, mode_kelas="sparse"):
        data_generator = ImageDataGenerator(rescale=1/255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')
        output = data_generator.flow_from_directory(input_directory, (input_img_size, input_img_size), class_mode=mode_kelas)
        return output

    def apply_preprocessing(dataset_folder,
                            output_folder,
                            val_split: float = 0.4,
                            apply_aug: bool = True):

        for folder in [output_folder, os.path.join(output_folder, 'training'), os.path.join(output_folder, 'validation')]:
            if not os.path.exists(folder):
                os.makedirs(folder)

    # Parameter untuk ukuran dataset validation
        validation_split = val_split
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

        # apply_augmentation
        def apply_augmentation(image):

            if random.random() < 0.5:
                # Flip gambar secara horizontal
                image = cv2.flip(image, 1)

            if random.random() < 0.5:
                # Menaikkan dan menurunkan kecerahan
                alpha = random.uniform(0.5, 1.5)
                image = cv2.multiply(image, np.array([alpha]))

            if random.random() < 0.5:
                # Menaikkan dan menurunkan kontras
                alpha = random.uniform(0.5, 1.5)
                grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                grey = cv2.cvtColor(grey, cv2.COLOR_GRAY2BGR)
                image = cv2.addWeighted(image, alpha, grey, 1 - alpha, 0)

            # if random.random() < 0.5:
            #     # Rotasi 90 derajat
            #     image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)

            return image

        for class_name in os.listdir(dataset_folder):
            class_folder = os.path.join(dataset_folder, class_name)

            training_class_folder = os.path.join(output_folder, 'training', class_name)
            validation_class_folder = os.path.join(output_folder, 'validation', class_name)
            for folder in [training_class_folder, validation_class_folder]:
                if not os.path.exists(folder):
                    os.makedirs(folder)

            image_files = os.listdir(class_folder)
            random.shuffle(image_files)
            num_validation_images = int(len(image_files) * validation_split)

            for i, image_name in enumerate(image_files) :
                image_path = os.path.join(class_folder, image_name)

                if i < num_validation_images:
                    output_image_path = os.path.join(validation_class_folder, image_name)
                else:
                    output_image_path = os.path.join(training_class_folder, image_name)

                image = cv2.imread(image_path)
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=16, minSize=(64, 64))

                if len(faces)>0:
                    for (x, y, w, h) in faces:
                        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 0), 0)
                        cropped = image[y:y + h, x:x + w]
                try:
                    cv2.imwrite(output_image_path, cropped)
                except:
                    continue

            if apply_aug == True:
                for i, image_name in enumerate(image_files) :
                    image_path = os.path.join(class_folder, image_name)
                    output_image_path = os.path.join(training_class_folder, image_name)
                    image = cv2.imread(image_path)
                    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=16, minSize=(64, 64))
                    if len(faces)>0:
                        for (x, y, w, h) in faces:
                            cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 0), 0)
                            cropped = image[y:y + h, x:x + w]
                    try:
                        augmented = apply_augmentation(cropped)
                        cv2.imwrite(output_image_path, augmented)
                    except:
                        continue

## Model

In [4]:
from keras.applications.inception_resnet_v2 import preprocess_input, InceptionResNetV2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [5]:
data_path = os.path.join("D:\CNN-Tim\dataset")
train_path = os.path.join(data_path, 'training')
validation_path = os.path.join(data_path, 'validation')

In [6]:
train_gen = ImageDataGenerator(preprocess_input)
validate_gen = ImageDataGenerator(preprocess_input)

In [7]:
train_gen = train_gen.flow_from_directory(train_path, 
                                      target_size = (224,224), 
                                      batch_size = 5)
validate_gen = validate_gen.flow_from_directory(validation_path, 
                                            target_size = (224,224), 
                                            batch_size = 5)

Found 246 images belonging to 11 classes.


Found 76 images belonging to 11 classes.


In [8]:
import tensorflow as tf

model1 = tf.keras.models.Sequential([
    # 1st conv
    tf.keras.layers.Conv2D(32, (5, 5), strides=(5, 5), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, strides=(2, 2)),  # Adjust strides here
    
    # 2nd conv
    tf.keras.layers.Conv2D(64, (11, 11), strides=(1, 1), activation='relu', padding="same"),  # Change strides to (1, 1)
    tf.keras.layers.BatchNormalization(),
    
    # 3rd conv
    tf.keras.layers.Conv2D(128, (11, 11), strides=(1, 1), activation='relu', padding="same"),  # Change strides to (1, 1)
    tf.keras.layers.BatchNormalization(),
    
    # 4th conv
    tf.keras.layers.Conv2D(64, (11, 11), strides=(1, 1), activation='relu', padding="same"),  # Change strides to (1, 1)
    tf.keras.layers.BatchNormalization(),
    
    # 5th Conv
    tf.keras.layers.Conv2D(32, (11, 11), strides=(1, 1), activation='relu', padding="same"),  # Change strides to (1, 1)
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.MaxPooling2D(2, strides=(2, 2)),  # Adjust strides here

    # To Flatten layer
    tf.keras.layers.Flatten(),
    
    # To FC layer 1
    tf.keras.layers.Dense(1024, activation='relu'),
    
    # Add dropout
    tf.keras.layers.Dropout(0.8),
    
    # To FC layer 2
    tf.keras.layers.Dense(3, activation='softmax')
])


In [14]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model

# Create the MobileNetV2 base model with its default input shape (224, 224, 3)
base_model = MobileNetV2(weights='imagenet', include_top=False)

# Set every layer in the base_model as non-trainable
for layer in base_model.layers:
    layer.trainable = False

# Create a new input layer with the desired input shape
new_input_shape = (224, 224, 3)
input_layer = tf.keras.layers.Input(shape=new_input_shape)

# Create your custom model (model1) and pass the input through it

model1 = tf.keras.models.Sequential([
    # 1st conv
    tf.keras.layers.Conv2D(32, (5, 5), strides=(5, 5), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, strides=(2, 2)),  # Adjust strides here
    
    # 2nd conv
    tf.keras.layers.Conv2D(64, (11, 11), strides=(1, 1), activation='relu', padding="same"),  # Change strides to (1, 1)
    tf.keras.layers.BatchNormalization(),
    
    # 3rd conv
    tf.keras.layers.Conv2D(128, (11, 11), strides=(1, 1), activation='relu', padding="same"),  # Change strides to (1, 1)
    tf.keras.layers.BatchNormalization(),
    
    # 4th conv
    tf.keras.layers.Conv2D(64, (11, 11), strides=(1, 1), activation='relu', padding="same"),  # Change strides to (1, 1)
    tf.keras.layers.BatchNormalization(),
    
    # 5th Conv
    tf.keras.layers.Conv2D(32, (11, 11), strides=(1, 1), activation='relu', padding="same"),  # Change strides to (1, 1)
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.MaxPooling2D(2, strides=(2, 2)),  # Adjust strides here

    # To Flatten layer
    tf.keras.layers.Flatten(),
    
    # To FC layer 1
    tf.keras.layers.Dense(1024, activation='relu'),
    
    # Add dropout
    tf.keras.layers.Dropout(0.8),
    
    # To FC layer 2
    tf.keras.layers.Dense(3, activation='softmax')
])


# Pass the input through model1
x = model1(input_layer)


# Add any additional custom layers as needed
x = Flatten()(x)
x = Dense(train_gen.num_classes, activation='softmax')(x)

# Create the final combined model
combined_model = Model(inputs=input_layer, outputs=x)
combined_model.summary()


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 sequential_5 (Sequential)   (None, 3)                 6451107   
                                                                 
 flatten_9 (Flatten)         (None, 3)                 0         
                                                                 
 dense_15 (Dense)            (None, 11)                44        
                                                                 
Total params: 6,451,151
Trainable params: 6,450,511
Non-trainable params: 640
_________________________________________________________________


In [15]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model

# Create the MobileNetV2 base model with its default input shape (224, 224, 3)
base_model = MobileNetV2(weights='imagenet', include_top=False)

# Set every layer in the base_model as non-trainable
for layer in base_model.layers:
    layer.trainable = False



# Combine your models

  # Set x to the input layer
model = Model(inputs=base_model.input, outputs=base_model.layers[-4].output)
base_out = base_model.output
base_out = Flatten()(base_out)
base_out = Dense(train_gen.num_classes, activation='softmax')(base_out)
model1 = Model(inputs=model.input, outputs=base_out)
model.summary()

ValueError: The last dimension of the inputs to a Dense layer should be defined. Found None. Full input shape received: (None, None)

In [ ]:
# base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(128,128,3))
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(train_gen.num_classes, activation='softmax')(x) 

# model = Model(inputs=base_model.input, outputs=x)